In [1]:
import requests as req
import pandas as pd
import sqlite3

In [2]:
ENDPOINT = 'https://api.hh.ru/vacancies'
URL_HEADERS = {'User-Agent': 'Mozilla/5.0'}
DB_NAME = 'hh.db'
TABLE_NAME = 'vacancies_api'
LIMIT_OF_VACANCY_DETAIL = 100

In [3]:

api_params = {'text':'python middle developer',
              'search_field':'name',
              'per_page': 25,
              'page':0}
vacancyDetailInfo =[]

In [4]:
def parsJsonAppendData (data):
    for result in data['items']:
        vacancyRow = []
        listOfRequirements = []
        vacancyName = result['name']
        companyName = result['employer']['name']
        vacancyid = result['id']
        vacancyDetail = req.get(ENDPOINT+'/'+vacancyid, headers = URL_HEADERS).json()
        print(ENDPOINT+'/'+vacancyid)
        vacancyDescription = vacancyDetail['description']
        for skill in vacancyDetail['key_skills']:
            listOfRequirements.append(skill['name'])
        stringOfRequirements = ', '.join(listOfRequirements)
        vacancyRow.append(companyName)
        vacancyRow.append(vacancyName)
        vacancyRow.append(vacancyDescription)
        vacancyRow.append(stringOfRequirements)
        vacancyDetailInfo.append(vacancyRow)

In [ ]:
results['items'][0]['name']

In [5]:
while (True):
    results = req.get(ENDPOINT, headers = URL_HEADERS, params = api_params).json()
    if (len(results['items']) == 0 or len(vacancyDetailInfo)>=LIMIT_OF_VACANCY_DETAIL):
        break
    parsJsonAppendData(results)
    print(f'Список детальной информации о вакансиях содержит {len(vacancyDetailInfo)} записей')
    api_params['page'] += 1
    

https://api.hh.ru/vacancies/82247456
https://api.hh.ru/vacancies/82440741
https://api.hh.ru/vacancies/81780083
https://api.hh.ru/vacancies/81949300
https://api.hh.ru/vacancies/82104952
https://api.hh.ru/vacancies/81941871
https://api.hh.ru/vacancies/82402579
https://api.hh.ru/vacancies/82260854
https://api.hh.ru/vacancies/82092062
https://api.hh.ru/vacancies/82172506
https://api.hh.ru/vacancies/81510302
https://api.hh.ru/vacancies/82055830
https://api.hh.ru/vacancies/82467015
https://api.hh.ru/vacancies/82119779
https://api.hh.ru/vacancies/82244830
https://api.hh.ru/vacancies/82119961
https://api.hh.ru/vacancies/82108779
https://api.hh.ru/vacancies/82306450
https://api.hh.ru/vacancies/82333777
https://api.hh.ru/vacancies/82333627
https://api.hh.ru/vacancies/81254369
https://api.hh.ru/vacancies/81826635
https://api.hh.ru/vacancies/81123242
https://api.hh.ru/vacancies/82056502
https://api.hh.ru/vacancies/81123241
Список детальной информации о вакансиях содержит 25 записей
https://api.hh.

In [6]:
table_columns = ['company_name', 'position', 'job_description', 'key_skills']
df = pd.DataFrame(data=vacancyDetailInfo,columns=table_columns,index=None)

In [8]:
connection = sqlite3.connect(DB_NAME)
df.to_sql(TABLE_NAME,connection,if_exists='replace',index=False)
connection.close()